In [2]:
import yfinance as yf
import pandas as pd
import numpy as np
from itertools import combinations
import warnings
warnings.filterwarnings('ignore')

# 28 Major FX pairs (Yahoo Finance tickers)
FX_PAIRS = [
    'EURUSD=X', 'GBPUSD=X', 'USDJPY=X', 'USDCHF=X', 'AUDUSD=X', 'USDCAD=X', 'NZDUSD=X',
    'EURGBP=X', 'EURJPY=X', 'EURCHF=X', 'EURCAD=X', 'EURAUD=X', 'EURNZD=X',
    'GBPJPY=X', 'GBPCHF=X', 'GBPCAD=X', 'GBPAUD=X', 'GBPNZD=X',
    'AUDJPY=X', 'AUDCHF=X', 'AUDCAD=X', 'AUDNZD=X',
    'NZDJPY=X', 'NZDCHF=X', 'NZDCAD=X', 'CADJPY=X', 'CADCHF=X', 'CHFJPY=X'
]


def fetch_fx_data(pairs, period='10d', interval='15m'):
    """Fetch OHLC data for FX pairs"""
    data = yf.download(pairs, period=period, interval=interval)['Close']
    data = data.dropna(axis=1, thresh=len(data)*0.8)  # Drop pairs with too many NaNs
    return data

def calculate_log_returns(prices):
    """Calculate log returns: ln(Pt/Pt-1)"""
    return np.log(prices / prices.shift(1)).dropna()

def correlation_matrix(returns):
    """Compute full correlation matrix"""
    return returns.corr()

def detect_decoupling_signals(pairs, high_corr_threshold=0.5, drop_threshold=0.1):
    """
    Detect pairs with high 10-day correlation that dropped recently (24h)
    Returns: list of (pair1, pair2, corr_10d, corr_24h)
    """
    # 10-day M15 data
    prices_10d = fetch_fx_data(pairs[:10], period='10d', interval='15m')  # Limit for speed
    returns_10d = calculate_log_returns(prices_10d)
    corr_10d = correlation_matrix(returns_10d)
    
    # 24-hour M15 data  
    prices_24h = fetch_fx_data(pairs[:10], period='1d', interval='15m')
    returns_24h = calculate_log_returns(prices_24h)
    corr_24h = correlation_matrix(returns_24h)
    
    signals = []
    for pair1, pair2 in combinations(prices_10d.columns, 2):
        corr_long = corr_10d.loc[pair1, pair2]
        corr_short = corr_24h.loc[pair1, pair2] if pair1 in corr_24h.index and pair2 in corr_24h.columns else np.nan
        
        if (abs(corr_long) > high_corr_threshold and 
            not np.isnan(corr_short) and 
            abs(corr_short) < (abs(corr_long) - drop_threshold)):
            signals.append((pair1, pair2, corr_long, corr_short))
    
    return signals

# RUN ANALYSIS
print("=== FULL CORRELATION MATRIX (10 days M15) ===")
prices = fetch_fx_data(FX_PAIRS, period='10d', interval='15m')
returns = calculate_log_returns(prices)
corr_matrix = correlation_matrix(returns)
print(corr_matrix.round(3))

print("\n=== DECOUPLING SIGNALS (Hedging Opportunities) ===")
signals = detect_decoupling_signals(FX_PAIRS)
for signal in signals:
    p1, p2, c10d, c24h = signal
    print(f"{p1} vs {p2}: 10d={c10d:.3f} → 24h={c24h:.3f} (DROP: {c10d-c24h:.3f})")


=== FULL CORRELATION MATRIX (10 days M15) ===


[*********************100%***********************]  28 of 28 completed

2 Failed downloads:
['AUDNZD=X']: ConnectionError('Failed to perform, curl: (7) Failed to connect to query2.finance.yahoo.com port 443 after 2017 ms: Could not connect to server. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')
['AUDCHF=X']: ConnectionError('Failed to perform, curl: (7) Failed to connect to query2.finance.yahoo.com port 443 after 2016 ms: Could not connect to server. See https://curl.se/libcurl/c/libcurl-errors.html first for more details.')


Ticker    AUDCAD=X  AUDJPY=X  AUDUSD=X  CADCHF=X  CADJPY=X  CHFJPY=X  \
Ticker                                                                 
AUDCAD=X     1.000     0.486     0.611    -0.009    -0.113    -0.002   
AUDJPY=X     0.486     1.000     0.578     0.313     0.723     0.502   
AUDUSD=X     0.611     0.578     1.000     0.084     0.116     0.035   
CADCHF=X    -0.009     0.313     0.084     1.000     0.519    -0.139   
CADJPY=X    -0.113     0.723     0.116     0.519     1.000     0.591   
CHFJPY=X    -0.002     0.502     0.035    -0.139     0.591     1.000   
EURAUD=X    -0.540    -0.695    -0.694    -0.247    -0.287     0.040   
EURCAD=X     0.356    -0.217    -0.011    -0.502    -0.558     0.042   
EURCHF=X     0.183     0.224     0.106     0.473     0.195    -0.305   
EURGBP=X    -0.018     0.001    -0.111    -0.074     0.015     0.137   
EURJPY=X     0.107     0.702     0.121     0.163     0.741     0.770   
EURNZD=X    -0.367    -0.409    -0.554    -0.151    -0.119     0

[*********************100%***********************]  10 of 10 completed
[*********************100%***********************]  10 of 10 completed

AUDUSD=X vs EURUSD=X: 10d=0.545 → 24h=0.390 (DROP: 0.156)
AUDUSD=X vs GBPUSD=X: 10d=0.566 → 24h=0.436 (DROP: 0.130)
